In [ ]:
# Download and extract data files
import os
import urllib.request
import zipfile

url = 'https://github.com/jsoma/2025-ds-dojo/raw/main/docs/02-pandas/02-estats-pandas-data.zip'
print(f'Downloading data from {url}...')
urllib.request.urlretrieve(url, '02-estats-pandas-data.zip')

print('Extracting 02-estats-pandas-data.zip...')
with zipfile.ZipFile('02-estats-pandas-data.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

os.remove('02-estats-pandas-data.zip')
print('✓ Data files extracted!')

Check the data types. What's wrong with the **Total population** columns?

It's because the columns are `2,924,000`. How can we correct that? There are two ways – the best approach involves fixing it while you're reading in the data.

The long column name of `A1302_Total population (15-64)[person]` is awkward and difficult to type, so let's rename the columns to be:

- `pop_15-64`
- `pop_65-over`

What prefecture has the largest number of 65+ population?

Let's only look at **2019 data**. Filter your dataset for 2019, and confirm it has 47 rows.

Now find the prefecture with the largest number of 65+ population.

Tokyo is just *big*, though, so we should probably adjust this to be **based on percentage**.

Create a new column called `total` that is the total population.

Create two new columns for the percentage of the prefecture that is 15-64 and 65 and over. Name them `pct_15-64` and `pct_65-over`. 

> It's fine to keep the percentage as 0.0-1.0, but you can make it 0-100 if you'd really like!

What prefecture has the highest population of 65+ population?

Let's make a **graph** of the percentage of each prefecture that is 65 and over.

Save the dataframe as `population-with-totals.csv`. **Check the file after you save it to make sure it doesn't have a weird empty column.**

## Bonus: Prefecture names in kana

Right now the content is **only in romaji**. It would be nice to have it in kana instead! I've created `prefecture_names.csv` with a mapping between the two.

Try to read it in: there's a issue you'll need to solve before it will work! You probably want to call it `df_pref`  so it doesn't overwrite your previous dataframe.

Merge your dataframes together, saving it as `merged`. Then graph again with the `name_jp` column.

The plot might not work because it uses non-Latin characters! You'll need to find a font that works.

Maybe you'll need to list fonts?

## Bonus: Interactive visualization

Read in the file again. In one cell, be sure to:

- Take care of the thousands, making sure the population is read in as numbers
- Rename the columns as we did above
- Create the `total` column
- Create the percentage columns

But this time we will NOT remove the pre-2022 data.

In [ ]:
df = pd.read_csv("estat-population.csv", thousands=',')

df = df.rename(columns={
    'A1302_Total population (15-64)[person]': 'pop_15-64',
    'A1303_Total population (65 and over)[person]': 'pop_65-over'
})

df['total'] = df['pop_15-64'] + df['pop_65-over']
df['pct_15-64'] = df['pop_15-64'] / df['total']
df['pct_65-over'] = df['pop_65-over'] / df['total']

df.head()

Confirm you have 2,256 rows.

In [ ]:
df.shape

Use ChatGPT to create a graphic to show you each prefecture's population over time. **On Thursday we will look at what the best approach to visualizing this data might be.**

In [ ]:
import altair as alt

alt.Chart(df).mark_line().encode(
    x='YEAR:O',  # Treat YEAR as ordinal for line plot
    y='pct_65-over:Q',  # Quantitative for percentage
    color='AREA:N',  # Color by AREA (categorical)
    tooltip=['AREA', 'YEAR', 'pct_65-over']  # Tooltip to show details on hover
).properties(
    title='Percentage of Population 65+ by Area Over Time'
).interactive()  # Enable zoom and pan

In [ ]:
highlight = alt.selection_single(
    fields=['AREA'],  # Selection is based on AREA
    nearest=True,  # Select the nearest line to the hover
    on='mouseover',  # Selection activates on hover
    empty='none'  # If no line is hovered, nothing is selected
)

base = alt.Chart(df).mark_line().encode(
    x='YEAR:O',
    y='pct_65-over:Q',
    color=alt.condition(highlight, 'AREA:N', alt.value('lightgray')),  # Highlight hovered line
    size=alt.condition(highlight, alt.value(3), alt.value(1)),  # Thicker line for the highlighted area
    tooltip=['AREA', 'YEAR', 'pct_65-over']
)

# Add points to make the hover interaction easier
points = base.mark_point().encode(
    opacity=alt.value(0)  # Invisible points, only for interaction purposes
).add_selection(
    highlight
)

# Combine the lines and points
chart = (base + points).properties(
    title='Percentage of Population 65+ by Area Over Time'
).interactive()

# Show the chart
chart

In [ ]:
base = alt.Chart(df).mark_line().encode(
    x='YEAR:O',
    y='pct_65-over:Q',
    tooltip=['AREA', 'YEAR', 'pct_65-over']
).properties(
    width=150,  # Set the width of each chart
    height=100  # Set the height of each chart
)

# Facet the chart into a grid where each AREA gets its own subplot
grid = base.facet(
    facet='AREA:N',  # Facet by AREA (each AREA gets a chart)
    columns=4  # Specify the number of columns in the grid
).properties(
    title='Percentage of Population 65+ by Area Over Time'
)  # Interactive zoom and pan

# Show the chart
grid


In [ ]:
df['pct_change'] = df.groupby('AREA')['pct_65-over'].pct_change()  # Calculate percentage change

In [ ]:
base = alt.Chart(df).mark_line().encode(
    x='YEAR:O',
    y='pct_change:Q',  # Use pct_change instead of pct_65-over
    tooltip=['AREA', 'YEAR', 'pct_change']
).properties(
    width=150,  # Set the width of each chart
    height=100  # Set the height of each chart
)

# Facet the chart into a grid where each AREA gets its own subplot
grid = base.facet(
    facet='AREA:N',  # Facet by AREA (each AREA gets a chart)
    columns=4  # Specify the number of columns in the grid
).properties(
    title='Percentage of Population 65+ by Area Over Time'
)  # Interactive zoom and pan

# Show the chart
grid